In [1]:
import os

os.environ['AZURE_SEARCH_SERVICE_ENDPOINT'] = 'https://chatwithdata505.search.windows.net'
os.environ['AZURE_SEARCH_API_KEY'] = 'eZpnhPvigeRZbuZ44usNcTzTwniL6xkPhJlalvrByzAzSeDC2TFk'

os.environ['OPENAI_API_KEY'] = '38ae4759658a4466b454666531283601'
os.environ['OPENAI_API_BASE'] = 'https://aoai-gpt4-505.openai.azure.com/'

search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = 'ifrs9'
search_api_key = os.environ["AZURE_SEARCH_API_KEY"]

In [2]:
from typing import Callable, Dict, List, Optional
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import logging
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

logger = logging.getLogger(__name__)

class AzureSearchRetrieveUserProxyAgent(RetrieveUserProxyAgent):
    def __init__(
        self,
        name="RetrieveChatAgent",
        human_input_mode: Optional[str] = "ALWAYS",
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        retrieve_config: Optional[Dict] = None,
        **kwargs,
    ):
        """
        Initialization for AzureSearchRetrieveUserProxyAgent with Azure Cognitive Search specifics.
        """
        super().__init__(name, human_input_mode, is_termination_msg, retrieve_config, **kwargs)
        self.endpoint = self._retrieve_config.get("endpoint")
        self.index_name = self._retrieve_config.get("index_name")
        self.api_key = self._retrieve_config.get("api_key")
        self._client = SearchClient(endpoint=self.endpoint,
                                    index_name=self.index_name,
                                    credential=AzureKeyCredential(self.api_key))

    def retrieve_docs(self, problem: str, n_results: int = 20, search_string: str = ""):
        """
        Retrieves documents from Azure Cognitive Search.
        """
        results = self.ai_search_query(problem=problem, n_results=n_results, search_string=search_string)
        self._results = results

    def ai_search_query(self, problem: str, n_results: int = 20, search_string: str = "") -> List[Dict]:
        """
        Perform a search query against Azure Cognitive Search.

        Args:
            problem (str): The problem to be solved.
            n_results (int): Number of results to return.
            search_string (str): A specific search string to filter results.

        Returns:
            List[Dict]: A list of dictionaries, each representing a document matching the search criteria.
        """
        # Construct search parameters
        search_params = {
            "search_text": problem,
            "include_total_count": True,
            "top": n_results,
        }
        if search_string:
            # You might want to adjust the filter syntax based on your Azure Cognitive Search index setup
            search_params["filter"] = f"document/any(d: d eq '{search_string}')"

        results = self._client.search(**search_params)
        
        ids, documents, metadatas, score, embeddings, uris = [], [], [], [], [], []
        for result in results:
            ids.append(result["id"])
            documents.append(result.get("content"))  # Assuming 'content' holds the document text
            metadatas.append({key: value for key, value in result.items() if key not in ["id", "content", "@search.score"]})
            score.append(result["@search.score"])
            embeddings.append(result.get("embedding"))
            uris.append(result.get("url"))


         # Formatting results as per the specified structure
        formatted_results = {
            'ids': [ids],  # Nested lists to match the specified structure
            'distances': [score], # Distances if available
            'metadatas': [metadatas],  # Metadata excluding the id and content
            'embeddings': [embeddings],  # Embeddings if available
            'documents': [documents],  # Actual document content
            'uris': [uris],  # URIs if available
            'data': None,  # Additional data placeholder if needed
        }

        return formatted_results


# Example usage
if __name__ == "__main__":
    azure_retrieve_config = {
        "endpoint": search_endpoint,
        "index_name": index_name,
        "api_key": search_api_key,
        "docs_path": None,
    }
    agent = AzureSearchRetrieveUserProxyAgent(retrieve_config=azure_retrieve_config)
    agent.retrieve_docs("what is sppi?")
    print(len(agent._results))
   
    # conver the results to pandas dataframe
    # import pandas as pd
    # df = pd.DataFrame(agent._results, columns=agent._results.keys())

    # print(df.head())



7
                                                 ids  \
0  [aHR0cHM6Ly9jaGF0ZGF0YTUwNS5ibG9iLmNvcmUud2luZ...   

                                           distances  \
0  [2.14224, 1.7972128, 1.686929, 1.3501315, 1.32...   

                                           metadatas  \
0  [{'title': '"\nApplying IFRS\n\nClassification...   

                                          embeddings  \
0  [None, None, None, None, None, None, None, Non...   

                                           documents  \
0  [\n\nDebt covenants\n\nQ31: Do debt covenants ...   

                                                uris data  
0  [https://chatdata505.blob.core.windows.net/fil...  NaN  


In [3]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-35-turbo",
            "gpt-3.5-turbo",
            "gpt-35-turbo-16k",
        }
    },
)

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-4', 'gpt-4-32k', 'gpt-35-turbo', 'gpt-35-turbo-16k']


In [4]:
azure_retrieve_config = {
        "endpoint": search_endpoint,
        "index_name": index_name,
        "api_key": search_api_key,
        "docs_path": None,
        "task": "qa"
    }

In [5]:
# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)

# 2. create the QdrantRetrieveUserProxyAgent instance named "ragproxyagent"
# By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
# `docs_path` is the path to the docs directory. It can also be the path to a single file, or the url to a single file. By default,
# it is set to None, which works only if the collection is already created.
#
# Here we generated the documentations from FLAML's docstrings. Not needed if you just want to try this notebook but not to reproduce the
# outputs. Clone the FLAML (https://github.com/microsoft/FLAML) repo and navigate to its website folder. Pip install and run `pydoc-markdown`
# and it will generate folder `reference` under `website/docs`.
#
# `task` indicates the kind of task we're working on. In this example, it's a `code` task.
# `chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
# We use an in-memory QdrantClient instance here. Not recommended for production.
# Get the installation instructions here: https://qdrant.tech/documentation/guides/installation/
ragproxyagent = AzureSearchRetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config=azure_retrieve_config,
    # code_execution_config={
    #     "use_docker": False,}
)

In [6]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "what is sppi?"
chat = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem, n_results=2)

Adding doc_id aHR0cHM6Ly9jaGF0ZGF0YTUwNS5ibG9iLmNvcmUud2luZG93cy5uZXQvaWZyczktY2h1bmtzL2FIUjBjSE02THk5amFHRjBaR0YwWVRVd05TNWliRzlpTG1OdmNtVXVkMmx1Wkc5M2N5NXVaWFF2Wm1sc1pYVndiRzloWkMxcFpuSnpPUzlsZVMxaGNIQnNlUzFtYVMxdFlYa3lNREUxTG5Ca1pnMi9jb250ZW50X2NodW5rc18zNC5qc29u0 to context.
Adding doc_id aHR0cHM6Ly9jaGF0ZGF0YTUwNS5ibG9iLmNvcmUud2luZG93cy5uZXQvaWZyczktY2h1bmtzL2FIUjBjSE02THk5amFHRjBaR0YwWVRVd05TNWliRzlpTG1OdmNtVXVkMmx1Wkc5M2N5NXVaWFF2Wm1sc1pYVndiRzloWkMxcFpuSnpPUzlsZVMxaGNIQnNlUzFtYVMxdFlYa3lNREUxTG5Ca1pnMi9jb250ZW50X2NodW5rc18zOC5qc29u0 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: what is sppi?

Context is: \n\nDebt covenants\n\nQ31: Do debt covenants result in cash flows that 

In [9]:
chat.chat_history

[{'content': "You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the\ncontext provided by the user.\nIf you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser's question is: what is sppi?\n\nContext is: \\n\\nDebt covenants\\n\\nQ31: Do debt covenants result in cash flows that are solely payments of\\nprincipal and interest on the principal amount outstanding?\\n\\nAdditional information\\n\\nA loan agreement contains a covenant whereby the contractual spread above\\nthe benchmark rate will increase if the borrower\\u2019s earnings before interest, tax,\\ndepreciation and amortisation (EBITDA) or its debt-to-equity ratio deteriorate\\nby a specified amount on a specified date.\\n\\nAnalysis\\n\\nWhether this instrument passes the SPPI test depends on the specific terms.\\n\\nThe loan would pass the SPPI test if the covenant serves t

In [8]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

ragproxyagent.human_input_mode = "ALWAYS"
qa_problem = "what is sppi?"
chat = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem, n_results=2)

Adding doc_id aHR0cHM6Ly9jaGF0ZGF0YTUwNS5ibG9iLmNvcmUud2luZG93cy5uZXQvaWZyczktY2h1bmtzL2FIUjBjSE02THk5amFHRjBaR0YwWVRVd05TNWliRzlpTG1OdmNtVXVkMmx1Wkc5M2N5NXVaWFF2Wm1sc1pYVndiRzloWkMxcFpuSnpPUzlsZVMxaGNIQnNlUzFtYVMxdFlYa3lNREUxTG5Ca1pnMi9jb250ZW50X2NodW5rc18zNC5qc29u0 to context.
Adding doc_id aHR0cHM6Ly9jaGF0ZGF0YTUwNS5ibG9iLmNvcmUud2luZG93cy5uZXQvaWZyczktY2h1bmtzL2FIUjBjSE02THk5amFHRjBaR0YwWVRVd05TNWliRzlpTG1OdmNtVXVkMmx1Wkc5M2N5NXVaWFF2Wm1sc1pYVndiRzloWkMxcFpuSnpPUzlsZVMxaGNIQnNlUzFtYVMxdFlYa3lNREUxTG5Ca1pnMi9jb250ZW50X2NodW5rc18zOC5qc29u0 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: what is sppi?

Context is: \n\nDebt covenants\n\nQ31: Do debt covenants result in cash flows that 